In [2]:
import numpy as np
import pandas as pd

In [3]:
# Inputs
energy_consumption_old = 2000 #KWh
energy_consumption_new = 1500 #KWh
bld_sq_footage = 210000 #sq.foot

In [4]:
energy_eff = ((energy_consumption_old - energy_consumption_new)/energy_consumption_old)
energy_eff

0.25

In [10]:
# Bonus Deduction Flags
prevailing_wage = 0
apprenticeship_hours = 0

## IRA Limitations

#### >ASHRAE standard in effect from 4 years prior to completion of construction
#### >Property capital cost value (installation cost) should not be greater IRA deduction
#### >Building modeling must be done by a contractor or P.E. licensed in the state where the building is located

## Bonus Deduction Criteria
#### >Meet the local prevailing wage (IRS)
#### >Meet apprenticeship percentage hours for up to 15% of labor hours (IRS)

## Extra Information
#### >A 3-year cap that allows IRC section 179D to be claimed on buildings if the previous full deduction claim occured more than 3 taxable years ago
#### >Owners/Tenants who made the installments can claim the tax rebate
#### https://www.youtube.com/watch?v=lOCQwn8OkC0

## IRA Deduction 179D

In [26]:
Section179D_params = {
    'Base_Deduction_High' : 1.00,
    'Base_Deduction_Low' : 0.50,
    'Base_Deduction_Inc' : 0.02,
    'Bonus_Deduction_High' : 5.00,
    'Bonus_Deduction_Low' : 2.50,
    'Bonus_Deduction_Inc' : 0.10,
    'Energy_Efficiency_Low' : 0.25,
    'Energy_Efficiency_High' : 0.50,
    'Prevailing_Wages_Law' : 1,
    'Apprenticeship_Hours_Met' : 1,
}

In [27]:
def IRA_179D (energy_eff, bld_sq_footage, Section179D_params):
    base_deduction = 0
    bonus_deduction = 0
    # Base Deduction 
    if energy_eff >= Section179D_params['Energy_Efficiency_High']:
        base_deduction = Section179D_params['Base_Deduction_High'] * bld_sq_footage
    elif energy_eff > Section179D_params['Energy_Efficiency_Low'] \
         and energy_eff < Section179D_params['Energy_Efficiency_High']:
        additional_percent = energy_eff - Section179D_params['Energy_Efficiency_Low']
        base_deduction = Section179D_params['Base_Deduction_Low'] * bld_sq_footage
        base_deduction = base_deduction + \
                        (Section179D_params['Base_Deduction_Inc'] * additional_percent * bld_sq_footage)
    else:
        base_deduction = Section179D_params['Base_Deduction_Low'] * bld_sq_footage
    
    # Bonus Deduction
    if (Section179D_params['Prevailing_Wages_Law'] == 1 and Section179D_params['Apprenticeship_Hours_Met'] == 1):
        if energy_eff >= Section179D_params['Energy_Efficiency_High']:
            bonus_deduction = Section179D_params['Bonus_Deduction_High'] * bld_sq_footage
        elif energy_eff > Section179D_params['Energy_Efficiency_Low'] \
            and energy_eff < Section179D_params['Energy_Efficiency_High']:
            additional_percent = energy_eff - Section179D_params['Energy_Efficiency_Low']
            bonus_deduction = Section179D_params['Bonus_Deduction_Low'] * bld_sq_footage
            bonus_deduction = bonus_deduction + \
                              (Section179D_params['Bonus_Deduction_Inc'] * additional_percent * bld_sq_footage)
        else:
            bonus_deduction = Section179D_params['Bonus_Deduction_Low'] * bld_sq_footage
    return (base_deduction, bonus_deduction)

In [29]:
print(IRA_179D(0.50, 210000, Section179D_params))

(210000.0, 1050000.0)


## ITC Calculation

In [65]:
labor_requirement = 1
domestic_content = 1
energy_community = 0

In [66]:
def ITC_vs_PTC(project_cost,kwh_generated,labor_requirement,domestic_content,energy_community):
    # ITC Calculation
    if labor_requirement == 1:
        itc_tax_credit = project_cost * 0.30
        if domestic_content == 1 and energy_community == 1:
            itc_tax_credit = itc_tax_credit + 2*(project_cost * 0.10)
        elif domestic_content == 0 and energy_community == 1:
            itc_tax_credit = itc_tax_credit + (project_cost * 0.10)
        elif domestic_content == 1 and energy_community == 0:
            itc_tax_credit = itc_tax_credit + (project_cost * 0.10)
    else:
        itc_tax_credit = project_cost * 0.06
        if domestic_content == 1 and energy_community == 1:
            itc_tax_credit = itc_tax_credit + 2*(project_cost * 0.02)
        elif domestic_content == 0 and energy_community == 1:
            itc_tax_credit = itc_tax_credit + (project_cost * 0.02)
        elif domestic_content == 1 and energy_community == 0:
            itc_tax_credit = itc_tax_credit + (project_cost * 0.02)
            
    # PTC Calculation
    if labor_requirement == 1:
        ptc_tax_credit = kwh_generated * 0.0275
        if domestic_content == 1 and energy_community == 1:
            ptc_tax_credit = ptc_tax_credit + 2*(kwh_generated * 0.003)
        elif domestic_content == 0 and energy_community == 1:
            ptc_tax_credit = ptc_tax_credit + (kwh_generated * 0.003)
        elif domestic_content == 1 and energy_community == 0:
            ptc_tax_credit = ptc_tax_credit + (kwh_generated * 0.003)
    else:
        ptc_tax_credit = kwh_generated * 0.0055
        if domestic_content == 1 and energy_community == 1:
            ptc_tax_credit = ptc_tax_credit + 2*(kwh_generated * 0.001)
        elif domestic_content == 0 and energy_community == 1:
            ptc_tax_credit = ptc_tax_credit + (kwh_generated * 0.001)
        elif domestic_content == 1 and energy_community == 0:
            ptc_tax_credit = ptc_tax_credit + (kwh_generated * 0.001)
    print("ITC Tax Credit: ",itc_tax_credit)
    print("PTC Tax Credit: ",ptc_tax_credit)
    
    # Check for which is greater (only 1 can be claimed)
    if itc_tax_credit >= ptc_tax_credit:
        return itc_tax_credit
    else:
        return ptc_tax_credit

In [67]:
print(ITC_vs_PTC(100000,2700000,labor_requirement,domestic_content,energy_community))

ITC Tax Credit:  40000.0
PTC Tax Credit:  82350.0
82350.0


## CPACE Finance

In [13]:
# Input Parameters
loan_tenure = 10 #years
property_value = 2000000 #dollars
project_cost = 1000000
periods = 12 # monthly

In [10]:
def CPACE_finance(loan_tenure, property_value, project_cost, periods):
    # If the project cost is greater than the property value, 20-30% of the property value is granted as the loan amount.
    # We have taken an average of 25% for test
    if project_cost >= property_value:
        loan_amount = 0.25 * property_value
    else:
        loan_amount = project_cost
        
    if loan_tenure <= 10:
        max_interest_rate = 0.055
        min_interest_rate = 0.05
        
    elif loan_tenure > 10 and loan_tenure <=15:
        max_interest_rate = 0.06
        min_interest_rate = 0.055
    else:
        max_interest_rate = 0.0625
        min_interest_rate = 0.06
    
    total_amount_max = loan_amount * (1 + (max_interest_rate/periods))**(periods*loan_tenure)
    total_interest_max = total_amount_max - loan_amount

    total_amount_min = loan_amount * (1 + (min_interest_rate/periods))**(periods*loan_tenure)
    total_interest_min = total_amount_min - loan_amount
    
    return total_interest_max, total_interest_min

In [11]:
max_int, min_int = CPACE_finance(loan_tenure, property_value, project_cost, periods)

In [12]:
print("Max Interest Amount : ",max_int)
print("Min Interest Amount : ",min_int)

Max Interest Amount :  731076.4170660307
Min Interest Amount :  647009.49769028


In [59]:
def solve(A):
        if A == 1:
            return str(A) + " "
        return solve(A - 1) + str(A) + " "

In [61]:
solve(18)

'1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 '